In [1]:
!pip install keras tensorflow

    100% |████████████████████████████████| 307kB 993kB/s ta 0:00:01
    100% |████████████████████████████████| 56.1MB 288kB/s ta 0:00:011    44% |██████████████▏                 | 24.7MB 1.5MB/s eta 0:00:22    77% |████████████████████████▉       | 43.5MB 1.3MB/s eta 0:00:10
    100% |████████████████████████████████| 51kB 125kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 1.7MB/s ta 0:00:011
    100% |████████████████████████████████| 1.2MB 1.4MB/s ta 0:00:011
    100% |████████████████████████████████| 3.3MB 730kB/s ta 0:00:01
    100% |████████████████████████████████| 2.0MB 1.2MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 1.4MB/s ta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /Users/jester/Library/Caches/pip/wheels/3c/33/ae/db8cd618e62f87594c13a5483f96e618044f9b01596efd013f
  Running setup.py bdist_wheel for termcolor ... done
  Stored in directory: /Users/jester/Library/Caches/pip/wheels/7c/06/54/bc84598

In [54]:
import cv2
import matplotlib.pyplot as plt
import random
import numpy as np
import pickle

from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras import backend
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Dropout

In [37]:
data = []
labels = []

img_paths = list(paths.list_images('datasets/animals/'))
random.shuffle(img_paths)

for img_path in img_paths :
    img = cv2.imread(img_path)
    img = cv2.resize(img, (32, 32))
    label = img_path.split('/')[-2]
    data.append(img)
    labels.append(label)

data = np.array(data)
labels = np.array(labels)

data = data / 255.0

In [38]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [39]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=33)

In [40]:
model = Sequential()

# create kernel
# activation ทำให้เป็น non linear
model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(32, 32, 3), 
        activation='relu',
        padding='same'
    )
)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 98307     
Total params: 99,203
Trainable params: 99,203
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [42]:
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.8942 - acc: 0.5646 - val_loss: 0.7188 - val_acc: 0.6667
Epoch 2/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.6296 - acc: 0.7208 - val_loss: 0.6702 - val_acc: 0.6896
Epoch 3/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.5201 - acc: 0.7802 - val_loss: 0.6942 - val_acc: 0.6813
Epoch 4/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.4248 - acc: 0.8495 - val_loss: 0.7544 - val_acc: 0.6104
Epoch 5/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.3712 - acc: 0.8656 - val_loss: 0.7223 - val_acc: 0.6875
Epoch 6/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.3054 - acc: 0.9068 - val_loss: 0.6992 - val_acc: 0.7167
Epoch 7/10
1920/1920 [==============================] - 2s 1ms/step - loss: 0.2526 - acc: 0.9333 - val_loss: 0.6897 - val_acc: 0.7188
Epoch 8/10
1920

In [43]:
lb.classes_

array(['cats', 'dogs', 'panda'], dtype='<U5')

In [45]:
y_pred = model.predict(x_test)
y_pred.argmax(axis=1)

print(
    classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_
    )
)

             precision    recall  f1-score   support

       cats       0.54      0.69      0.61       204
       dogs       0.54      0.40      0.46       210
      panda       0.83      0.81      0.82       186

avg / total       0.63      0.63      0.62       600



In [51]:
# 32 คือ จำนวน kernal
model = Sequential()
model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(32, 32, 3), 
        activation='relu',
        padding='same'
    )
)
model.add(
    Conv2D(
        128, 
        (3, 3), 
        activation='relu',
        padding='same'
    )
)
model.add(
    Conv2D(
        128, 
        (3, 3), 
        activation='relu',
        padding='same'
    )
)

# ใช้ค่ามากสุดตอนทาบ kernal
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [52]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [53]:
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

Train on 1920 samples, validate on 480 samples
Epoch 1/10
1920/1920 [==============================] - 55s 29ms/step - loss: 0.8498 - acc: 0.5438 - val_loss: 0.7234 - val_acc: 0.6479
Epoch 2/10
1920/1920 [==============================] - 58s 30ms/step - loss: 0.6857 - acc: 0.6630 - val_loss: 0.7492 - val_acc: 0.6458
Epoch 3/10
1920/1920 [==============================] - 58s 30ms/step - loss: 0.6475 - acc: 0.6990 - val_loss: 0.8366 - val_acc: 0.5833
Epoch 4/10
1920/1920 [==============================] - 59s 31ms/step - loss: 0.5867 - acc: 0.7224 - val_loss: 0.7442 - val_acc: 0.6625
Epoch 5/10
1920/1920 [==============================] - 60s 31ms/step - loss: 0.5119 - acc: 0.7599 - val_loss: 0.6259 - val_acc: 0.7271
Epoch 6/10
1920/1920 [==============================] - 57s 30ms/step - loss: 0.4558 - acc: 0.8000 - val_loss: 0.6364 - val_acc: 0.7167
Epoch 7/10
1920/1920 [==============================] - 62s 32ms/step - loss: 0.3950 - acc: 0.8250 - val_loss: 0.6975 - val_acc: 0.6958
E

In [55]:
y_pred = model.predict(x_test)
y_pred.argmax(axis=1)

print(
    classification_report(
    y_test.argmax(axis=1), 
    y_pred.argmax(axis=1), 
    target_names=lb.classes_
    )
)

             precision    recall  f1-score   support

       cats       0.64      0.55      0.59       204
       dogs       0.55      0.64      0.59       210
      panda       0.87      0.83      0.85       186

avg / total       0.68      0.67      0.67       600



In [56]:
model.save('mymodel.h5')

In [57]:
f = open('lb.pkl', 'wb')
pickle.dump(lb, f)
f.close()